In [44]:
#Gathering Data
import os
import cv2

base_dir = 'C:/Users/ASUS RYZEN/AnacondaJupyter/Masker/DatasetCombinedv2'

mask_dir = os.path.join(base_dir, 'WithMask')
wout_dir = os.path.join(base_dir, 'WithoutMask')
    
mask_img_dir = list()
wout_img_dir = list()

for dirpath, dirnames, filenames in os.walk(mask_dir):
    for file in filenames:
        if ".png" in file:
            image_path = os.path.join(dirpath, file)
            mask_img_dir.append(image_path)

for dirpath, dirnames, filenames in os.walk(wout_dir):
    for file in filenames:
        if ".png" in file:
            image_path = os.path.join(dirpath, file)
            wout_img_dir.append(image_path)

In [45]:
#Resize + Save
from random import shuffle

target_dir = 'C:/Users/ASUS RYZEN/AnacondaJupyter/Masker/DatasetCombinedResized'


for i, img_path in enumerate(mask_img_dir):
    image = cv2.imread(img_path)
    image = cv2.resize(image, (300,300))

    file_name = f"mask.png"
    dest = os.path.join(target_dir, file_name)
    
    cv2.imwrite(dest, image)

for i, img_path in enumerate(wout_img_dir):
    image = cv2.imread(img_path)
    image = cv2.resize(image, (300,300))

    file_name = f"wout.png"
    dest = os.path.join(target_dir, file_name)
    
    cv2.imwrite(dest, image)

In [46]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from pathlib import Path
from PIL import Image
import numpy as np

In [47]:
base_model = DenseNet121()
model = Model(inputs=base_model.input,
                   outputs=base_model.get_layer('predictions').output)

model.save('Build_Modell_IR.h5')

In [48]:
class FeatureExtractor:
    def __init__(self, model):
        self.model = model
    
    def extract(self, img):
        img = img.resize((224, 224))
        img = img.convert('RGB')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        feature = self.model.predict(x)[0]
        
        return feature / np.linalg.norm(feature)

fe = FeatureExtractor(load_model('C:/Users/ASUS RYZEN/AnacondaJupyter/Masker/Build_Model_IR.h5', compile=False))

In [49]:
base_dir = 'C:/Users/ASUS RYZEN/AnacondaJupyter/Masker/DatasetCombinedResized'
feature_dir = 'C:/Users/ASUS RYZEN/AnacondaJupyter/Masker/feature_extract'

img_paths = list()
features = list()

for img_path in sorted(Path(base_dir).glob("*.png")):
    img_paths.append(img_path)
    feature = fe.extract(img=Image.open(img_path))
    features.append(feature)

features = np.array(features)
np.save(feature_dir + '/extracted_feature.npy', features)